# Model training lab

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

## Prerequisits

### Imports

In [1]:
# Imports
import os
import warnings
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import Bidirectional
from matplotlib import pyplot
from data_repository import DataRepository
# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load Data

In [2]:
# Root CSV files directory
dirname = "./data/absolute/2D/"  

repo = DataRepository(dirname, verbose=False)


#load all
x, y = repo.getDataAndLabels()

# Load data and print summary, if desired
x_train, x_val, x_test, y_train, y_val, y_test, labels = repo.getForTraining()

Amount Datasets by word total:
Computer :  58;  Deutschland :  67;  Haben :  71;  Hallo :  62;  Mainz :  67;  Software :  68;  Welt :  68;  du :  70;  ich :  38;  unser :  67;  zeigen :  70;   
Amount Datasets by word training:
Computer :  35;  Deutschland :  40;  Haben :  42;  Hallo :  37;  Mainz :  40;  Software :  41;  Welt :  41;  du :  42;  ich :  23;  unser :  40;  zeigen :  42;   
Amount Datasets by word validiation:
Computer :  12;  Deutschland :  13;  Haben :  14;  Hallo :  13;  Mainz :  14;  Software :  13;  Welt :  13;  du :  14;  ich :  8;  unser :  13;  zeigen :  14;   
Amount Datasets by word test:
Computer :  11;  Deutschland :  14;  Haben :  15;  Hallo :  12;  Mainz :  13;  Software :  14;  Welt :  14;  du :  14;  ich :  7;  unser :  14;  zeigen :  14;   
Distribution of data:
Amount total: 706
Amount training: 423
Amount validiation: 141
Amount test: 142
Tokens:
{'computer': 1, 'deutschland': 2, 'du': 3, 'haben': 4, 'hallo': 5, 'ich': 6, 'mainz': 7, 'software': 8, 'uns

In [3]:
print(x_train[1])

[[ 0.496078  0.242033  0.372475 ...  0.5       0.5       0.5     ]
 [ 0.495348  0.244273  0.370611 ...  0.262506  0.312994 -0.11808 ]
 [ 0.497854  0.244501  0.380122 ...  0.5       0.5       0.5     ]
 ...
 [ 0.5       0.5       0.5      ...  0.5       0.5       0.5     ]
 [ 0.5       0.5       0.5      ...  0.5       0.5       0.5     ]
 [ 0.5       0.5       0.5      ...  0.5       0.5       0.5     ]]


In [4]:
import winsound
def finished(num):
    frequency = 2000  # Set Frequency To 2500 Hertz
    duration = 500  # Set Duration To 1000 ms == 1 second
    for i in range(0, num):
        winsound.Beep(frequency, duration)

## Training Stage
Configure the model and train it.

Metrics:
<div float="right">
    <img src="assets/accuracy.png" width="400"> 
    <img src="assets/precision_recall_formula.png" width="400">
</div>
<img src="assets/precision_recall.png" width="1000">


### <span style="color:blue"> Hyperparametertuned LSTM </span>
##### Here it is necessary to install the Keras-Tuner Module by executing:
#####  <span style="color:green"> via Conda:</span>
conda install -c conda-forge keras-tuner
#####  <span style="color:green"> for pip:</span>
pip install keras-tuner

Right now there are three different builds we are testing:
- classic LSTM
- CuDNNLSTM
- bidriectional LSTM


In [8]:
from kerastuner.tuners import RandomSearch
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
from time import time, strftime


starttime= strftime("%Y_%m_%d_%H%M%S")
LOG_DIR = "C:\ML\Optimization_"f"{starttime}" #<-In Windows below Log_dir Path will maybe be too long for Windows to handle, so use a shorter path like this here
#LOG_DIR = "./Optimization_"f"{starttime}" # LOG_DIR holds json files with information and a model of each single trial

def build_model_lstm(hp):
    model = Sequential()
    
    model.add(layers.LSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64), #kerastuner will randomly choose a value for nodes between 128 and 256 in steps of 64
                            return_sequences=True,
                            input_shape=(x_train.shape[1], x_train.shape[2])))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(layers.LSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True))
    
    model.add(layers.LSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32)))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adam','RMSprop','SGD']),
                  optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax','Adam','RMSprop']),
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model



def build_model_CuDNNLSTM(hp):
    model = Sequential()
    

    
    model.add(tf.compat.v1.keras.layers.CuDNNLSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64), #kerastuner will randomly choose a value for nodes between 128 and 256 in steps of 64
                            return_sequences=True,
                            input_shape=(x_train.shape[1], x_train.shape[2])))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(tf.compat.v1.keras.layers.CuDNNLSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True))
    
    model.add(tf.compat.v1.keras.layers.CuDNNLSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32)))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adam','RMSprop','SGD']),
                  optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax','Adam','RMSprop']),
                  metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model



def build_model_bdlstm(hp):
    model = Sequential()
    model.add(Bidirectional(layers.LSTM(hp.Int("LSTM_input", min_value =64, max_value=256,step=64, default=64),
                                        return_sequences=True),
                                        input_shape=(x_train.shape[1], x_train.shape[2])))
    
    for i in range(hp.Int("n_layers" , 1, 3)):    #number of layers ramdom between 1 an 3
        model.add(layers.Bidirectional(layers.LSTM(hp.Int(f"LSTM_{i}_units", min_value =64, max_value=256,step=64, default=64),return_sequences=True)))
    
    model.add(layers.Bidirectional(layers.LSTM(hp.Int(f"LSTM_End", min_value =32, max_value=128,step=32, default=32))))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax','Adam','RMSprop']),
                  optimizer=hp.Choice('optimizer',values=['Adamax']),
                  metrics=['accuracy']) 
    model.summary()
    print(model.optimizer.get_config()["name"])
    print('')
    return model






###   <span style="color:red">Necesarry only in case of using Nvidia GPU  </span>

In [6]:
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs: 1


# Different Keras-Tuner Approaches
### 1 - RandomSearch
Parameter of variables are ranomly used (number of layers, number of nodes) and "best" model is chosen.

In [9]:
tuner  = RandomSearch(
    build_model_lstm,     #Function to use search in... See different builds above
    objective = "val_accuracy",  #Chooses "best model" looking for highest value of val_accuracy
    max_trials = 30,       # Number of different combinations tried Nodes and layers
    executions_per_trial = 1, 
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

#tuner.search_space_summary()

tuner.search(x=x_train,      #syntax just like in fit
                y= y_train,
            epochs=200,
            batch_size=32,
            validation_data=(x_val,y_val),
            verbose=2
            )

print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())

finished(8)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 64)           49408     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 64)           33024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense (Dense)                (None, 12)                396       
Total params: 95,244
Trainable params: 95,244
Non-trainable params: 0
_________________________________________________________________
Adagrad

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 192)          246528    
_______________________

Epoch 34/200
423/423 - 0s - loss: 1.5847 - accuracy: 0.3948 - precision: 0.6542 - recall: 0.1655 - val_loss: 1.6289 - val_accuracy: 0.3404 - val_precision: 0.5862 - val_recall: 0.1206
Epoch 35/200
423/423 - 0s - loss: 1.5862 - accuracy: 0.3593 - precision: 0.6386 - recall: 0.1253 - val_loss: 1.6741 - val_accuracy: 0.3262 - val_precision: 0.6071 - val_recall: 0.1206
Epoch 36/200
423/423 - 0s - loss: 1.4952 - accuracy: 0.4184 - precision: 0.6765 - recall: 0.1631 - val_loss: 1.4914 - val_accuracy: 0.3759 - val_precision: 0.5641 - val_recall: 0.1560
Epoch 37/200
423/423 - 0s - loss: 1.4869 - accuracy: 0.4066 - precision: 0.6514 - recall: 0.1678 - val_loss: 1.5015 - val_accuracy: 0.3617 - val_precision: 0.6190 - val_recall: 0.1844
Epoch 38/200
423/423 - 0s - loss: 1.4608 - accuracy: 0.3972 - precision: 0.6724 - recall: 0.1844 - val_loss: 1.6158 - val_accuracy: 0.3830 - val_precision: 0.6250 - val_recall: 0.1418
Epoch 39/200
423/423 - 0s - loss: 1.4738 - accuracy: 0.4208 - precision: 0.6508 

Epoch 79/200
423/423 - 0s - loss: 0.8103 - accuracy: 0.6974 - precision: 0.8097 - recall: 0.5934 - val_loss: 1.1119 - val_accuracy: 0.6028 - val_precision: 0.6531 - val_recall: 0.4539
Epoch 80/200
423/423 - 0s - loss: 0.8083 - accuracy: 0.7045 - precision: 0.8176 - recall: 0.5934 - val_loss: 0.7714 - val_accuracy: 0.7021 - val_precision: 0.7944 - val_recall: 0.6028
Epoch 81/200
423/423 - 0s - loss: 0.7062 - accuracy: 0.7139 - precision: 0.8125 - recall: 0.6147 - val_loss: 0.8446 - val_accuracy: 0.6454 - val_precision: 0.7843 - val_recall: 0.5674
Epoch 82/200
423/423 - 0s - loss: 0.6668 - accuracy: 0.7305 - precision: 0.8249 - recall: 0.6572 - val_loss: 1.4265 - val_accuracy: 0.5248 - val_precision: 0.5816 - val_recall: 0.4043
Epoch 83/200
423/423 - 0s - loss: 0.9742 - accuracy: 0.6785 - precision: 0.7855 - recall: 0.5887 - val_loss: 0.8070 - val_accuracy: 0.7163 - val_precision: 0.8211 - val_recall: 0.5532
Epoch 84/200
423/423 - 0s - loss: 0.6910 - accuracy: 0.7258 - precision: 0.8548 

Epoch 124/200
423/423 - 0s - loss: 0.6498 - accuracy: 0.7825 - precision: 0.8046 - recall: 0.7400 - val_loss: 0.7967 - val_accuracy: 0.7305 - val_precision: 0.7557 - val_recall: 0.7021
Epoch 125/200
423/423 - 0s - loss: 0.4095 - accuracy: 0.8582 - precision: 0.8942 - recall: 0.8392 - val_loss: 0.4824 - val_accuracy: 0.8723 - val_precision: 0.8846 - val_recall: 0.8156
Epoch 126/200
423/423 - 0s - loss: 0.3478 - accuracy: 0.9196 - precision: 0.9332 - recall: 0.8913 - val_loss: 0.5964 - val_accuracy: 0.8156 - val_precision: 0.8261 - val_recall: 0.8085
Epoch 127/200
423/423 - 0s - loss: 0.2365 - accuracy: 0.9338 - precision: 0.9379 - recall: 0.9291 - val_loss: 0.6499 - val_accuracy: 0.8440 - val_precision: 0.8561 - val_recall: 0.8440
Epoch 128/200
423/423 - 0s - loss: 0.2280 - accuracy: 0.9291 - precision: 0.9444 - recall: 0.9243 - val_loss: 0.8656 - val_accuracy: 0.7447 - val_precision: 0.7574 - val_recall: 0.7305
Epoch 129/200
423/423 - 0s - loss: 0.4064 - accuracy: 0.8794 - precision: 0

Epoch 169/200
423/423 - 0s - loss: 0.1161 - accuracy: 0.9693 - precision: 0.9691 - recall: 0.9645 - val_loss: 0.6654 - val_accuracy: 0.8369 - val_precision: 0.8357 - val_recall: 0.8298
Epoch 170/200
423/423 - 0s - loss: 0.2214 - accuracy: 0.9480 - precision: 0.9477 - recall: 0.9433 - val_loss: 0.6683 - val_accuracy: 0.8652 - val_precision: 0.8643 - val_recall: 0.8582
Epoch 171/200
423/423 - 0s - loss: 0.0582 - accuracy: 0.9811 - precision: 0.9857 - recall: 0.9811 - val_loss: 0.8178 - val_accuracy: 0.7730 - val_precision: 0.7770 - val_recall: 0.7660
Epoch 172/200
423/423 - 0s - loss: 0.1375 - accuracy: 0.9693 - precision: 0.9714 - recall: 0.9622 - val_loss: 0.5702 - val_accuracy: 0.8582 - val_precision: 0.8759 - val_recall: 0.8511
Epoch 173/200
423/423 - 0s - loss: 0.1696 - accuracy: 0.9574 - precision: 0.9596 - recall: 0.9551 - val_loss: 0.7076 - val_accuracy: 0.8227 - val_precision: 0.8273 - val_recall: 0.8156
Epoch 174/200
423/423 - 0s - loss: 0.0977 - accuracy: 0.9716 - precision: 0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 192)          246528    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 128)          164352    
_________________________________________________________________
lstm_2 (LSTM)                (None, 96)                86400     
_________________________________________________________________
dense (Dense)                (None, 12)                1164      
Total params: 498,444
Trainable params: 498,444
Non-trainable params: 0
_________________________________________________________________
Adagrad

Train on 423 samples, validate on 141 samples
Epoch 1/200
423/423 - 6s - loss: 2.4562 - accuracy: 0.0898 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 2.4354 - val_accuracy: 0.0993 - val_precision: 0.0000e+00 - val_recall: 0.000

Epoch 40/200
423/423 - 0s - loss: 1.3164 - accuracy: 0.5366 - precision: 0.7742 - recall: 0.1702 - val_loss: 1.3611 - val_accuracy: 0.4539 - val_precision: 0.7273 - val_recall: 0.1702
Epoch 41/200
423/423 - 0s - loss: 1.2942 - accuracy: 0.5272 - precision: 0.6860 - recall: 0.1962 - val_loss: 1.2760 - val_accuracy: 0.5319 - val_precision: 0.7209 - val_recall: 0.2199
Epoch 42/200
423/423 - 0s - loss: 1.2709 - accuracy: 0.5319 - precision: 0.7323 - recall: 0.2199 - val_loss: 2.1628 - val_accuracy: 0.3191 - val_precision: 0.5556 - val_recall: 0.1064
Epoch 43/200
423/423 - 0s - loss: 1.6130 - accuracy: 0.4184 - precision: 0.7300 - recall: 0.1726 - val_loss: 1.3618 - val_accuracy: 0.4681 - val_precision: 0.6970 - val_recall: 0.1631
Epoch 44/200
423/423 - 0s - loss: 1.2676 - accuracy: 0.5579 - precision: 0.7227 - recall: 0.2033 - val_loss: 1.2488 - val_accuracy: 0.5461 - val_precision: 0.7073 - val_recall: 0.2057
Epoch 45/200
423/423 - 0s - loss: 1.2079 - accuracy: 0.5697 - precision: 0.7879 

KeyboardInterrupt: 

### 2 - Hyperband
Variation of RandomSearch http://jmlr.org/papers/volume18/16-558/16-558.pdf

In [ ]:
tuner  = Hyperband(
    build_model,
    objective = "val_accuracy",
    hyperband_iterations=2,
    max_epochs=150,
    directory = LOG_DIR,
    project_name='SignLagnuageModelOptimization'
    )

#tuner.search_space_summary()

tuner.search(x=x_train, 
            y= y_train,
            batch_size=32,
            validation_data=(x_val,y_val))

print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())

finished(8)

In [ ]:
#Laut Randomsearch bestes Model am 23.06.2020

model = Sequential()
model.add(layers.LSTM(128, return_sequences=True,
               input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(layers.LSTM(64, return_sequences=True)) 
model.add(layers.LSTM(96))  
model.add(layers.Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
model.summary()

history=model.fit(x_train,y_train,epochs=170,validation_data=(x_val,y_val),shuffle=False,verbose=2)

In [ ]:
model = Sequential()
model.add(Bidirectional(layers.LSTM(256, return_sequences=True),
                                        input_shape=(x_train.shape[1], x_train.shape[2])))
    
model.add(layers.Bidirectional(layers.LSTM(192,return_sequences=True)))
    
model.add(layers.Bidirectional(layers.LSTM(64)))
model.add(layers.Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy',
                  #optimizer=hp.Choice('optimizer',values=['Adagrad','Adamax','Adam','RMSprop']),
                  optimizer='Adamax',
                  metrics='accuracy') 
model.summary()
#history=model.fit(x_train,y_train,epochs=170,validation_data=(x_val,y_val),shuffle=False,verbose=2)

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

## Export tuner object into pickle file
so it can be used in other scripts

In [ ]:
import pickle

with open(f"tuner_"f"{starttime}.pkl", "wb") as f:
    pickle.dump(tuner, f)
    

## Get best Trial from Tuner Object

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
bestmodel= tuner.hypermodel.build(best_hp)

bestmodel.summary()


In [ ]:
#tmp_chekpoints= "tmp\epoch{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5"
tmp_chekpoints= "C:\\ML\\checkpoints\\tmp\\epoch{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}.hdf5"

#csv_log = tf.keras.callbacks.CSVLogger("log.csv", separator=',', append=False)
csv_log = tf.keras.callbacks.CSVLogger("C:\ML\logs\log.csv", separator=',', append=False)

#tb = tf.keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=1, write_graph=False, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)
tb = tf.keras.callbacks.TensorBoard(log_dir='C:\ML\logs', histogram_freq=1, write_graph=False, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=1, embeddings_metadata=None)
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=20, verbose=0, mode='max', baseline=None, restore_best_weights=True)
chk= tf.keras.callbacks.ModelCheckpoint(tmp_chekpoints, monitor='val_accuracy', verbose=0, save_best_only=False, save_weights_only=False, mode='max', save_freq='epoch')


history = model.fit(x_train,y_train,epochs=200,batch_size=32, validation_data=(x_val,y_val),shuffle=False, verbose=2, callbacks=[csv_log, chk])

### Diagnostic Plots

The training history of your LSTM models can be used to diagnose the behavior of your model.

You can plot the performance of your model using the Matplotlib library. For example, you can plot training loss vs test loss as follows:

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.savefig("C:/ML/loss"f"{starttime}.png")
pyplot.show()

pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model train vs validation accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='lower right')
pyplot.savefig("C:/ML/accuracy_"f"{starttime}.png")
pyplot.show()

#### Underfit Example
Running this example produces a plot of train and validation loss showing the characteristic of an underfit model. In this case, performance may be improved by increasing the number of training epochs.


<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model.png" width="400">


Running this example shows the characteristic of an underfit model that appears under-provisioned.
In this case, performance may be improved by increasing the capacity of the model, such as the number of memory cells in a hidden layer or number of hidden layers.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Underfit-Model-via-Status.png" width="400">

#### Good Fit Example
Running the example creates a line plot showing the train and validation loss meeting.
Ideally, we would like to see model performance like this if possible, although this may not be possible on challenging problems with a lot of data.

<img src="assets/Diagnostic-Line-Plot-Showing-a-Good-Fit-for-a-Model.png" width="400">

#### Overfit Example
Running this example creates a plot showing the characteristic inflection point in validation loss of an overfit model.
This may be a sign of too many training epochs.
In this case, the model training could be stopped at the inflection point. Alternately, the number of training examples could be increased.

<img src="assets/Diagnostic-Line-Plot-Showing-an-Overfit-Model.png" width="400">

### Evaluate

In [ ]:
#model = tf.keras.models.load_model('./tmp/epoch49-0.90-0.39.hdf5')


#bestmodel.evaluate(x=x_test, y=y_test, verbose=2)
model.evaluate(x=x_test, y=y_test, verbose=2)


### Save model

In [ ]:
bestmodel.save("sign_lang_recognition_tuned.h5")